# Imports

In [53]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from lu_crtp import lu_crtp, compareApprox

# Define Data

In [122]:
item_user_data = np.array([[0,1,0,1,2,2],
              [2,3,1,1,2,2],
              [1,1,1,0,1,1],
              [0,2,3,4,1,1],
              [0,0,0,0,1,0]])

V = pd.DataFrame(item_user_data, index=['Vegetables', 'Fruits', 'Sweets', 'Bread', 'Coffee'])
V.columns = ['John', 'Alice', 'Mary', 'Greg', 'Peter', 'Andrew']

# Recommender System with no Dim. Reduction

In [135]:
from sklearn.decomposition import NMF

# Run NMF
nmf = NMF(3)
nmf.fit(V)

H = pd.DataFrame(np.round(nmf.components_,2), columns=V.columns)
H.index = ['Fruits pickers', 'Bread eaters',  'Veggies']

print(H)

W = pd.DataFrame(np.round(nmf.transform(V),2), columns=H.index)
W.index = V.index

print(W)

# Reconstruct original data
reconstructed = pd.DataFrame(np.round(np.dot(W,H),2), columns=V.columns)
reconstructed.index = V.index

                John  Alice  Mary  Greg  Peter  Andrew
Fruits pickers   0.0   2.43  4.66  5.58   0.00    0.16
Bread eaters     0.0   0.87  0.00  0.85   1.92    1.74
Veggies          1.5   1.71  0.80  0.15   0.76    0.83
            Fruits pickers  Bread eaters  Veggies
Vegetables            0.01          1.09     0.00
Fruits                0.04          0.55     1.33
Sweets                0.00          0.18     0.67
Bread                 0.64          0.52     0.00
Coffee                0.00          0.23     0.00


In [124]:
mse = mean_squared_error(V, reconstructed)
mse

0.03622333333333334

# Recommender System with PCA

In [125]:
# Standardize the data (optional)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
user_item_matrix_std = scaler.fit_transform(item_user_data)

# Apply PCA
pca = PCA(n_components=2)  # Reduce to 2 dimensions for simplicity
user_item_matrix_reduced = pca.fit_transform(user_item_matrix_std)

In [126]:
# Make sure Matrix is non-negative
min_value = np.min(user_item_matrix_reduced)
user_item_matrix_reduced_shifted = user_item_matrix_reduced - min_value

In [127]:
# Build a simple collaborative filtering model (e.g., using matrix factorization)
from sklearn.decomposition import NMF
nmf = NMF(n_components=2, init='random', random_state=42)
W = nmf.fit_transform(user_item_matrix_reduced_shifted)
H = nmf.components_

# Predict ratings
predicted_ratings = np.dot(W, H)

# Shift the output back to original form
predicted_ratings += min_value
predicted_ratings_original_space = pca.inverse_transform(predicted_ratings)

In [128]:
mse = mean_squared_error(item_user_data, predicted_ratings_original_space)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 1.5645170957314154


# Recommender System Using LU_CRTP

In [136]:
p_r, p_c, l_k, u_k, r_k = lu_crtp(item_user_data, 3)

In [139]:
l_k.shape

(5, 3)

In [140]:
u_k.shape

(3, 6)

In [133]:
l_k_df = pd.DataFrame(l_k, index=['Vegetables', 'Fruits', 'Sweets', 'Bread', 'Coffee'])

In [134]:
l_k_df

,0,1,2
Vegetables,1.000000,0.000000,0.000000
Fruits,0.000000,1.000000,0.000000
Sweets,0.000000,0.000000,1.000000
Bread,-0.142857,0.357143,0.214286
Coffee,-0.142857,-0.142857,0.714286


In [130]:
lu_reduced = np.dot(l_k, u_k)

In [116]:
lu_reduced_df = pd.DataFrame(lu_reduced, index=['John', 'Alice', 'Mary', 'Greg', 'Peter'])
lu_reduced_df.columns = ['Vegetables', 'Fruits', 'Sweets', 'Bread', 'Coffee', 'Cereal']

In [117]:
lu_reduced_df

,Vegetables,Fruits,Sweets,Bread,Coffee,Cereal
John,4.000000e+00,1.0,2.0,3.000000,1.0,0.000000
Alice,1.000000e+00,2.0,3.0,1.000000,2.0,2.000000
Mary,1.000000e+00,2.0,1.0,0.000000,2.0,0.000000
Greg,-5.551115e-17,1.0,1.0,-0.071429,1.0,0.714286
Peter,0.000000e+00,1.0,0.0,-0.571429,1.0,-0.285714


In [118]:
mse = mean_squared_error(item_user_data, lu_reduced_df)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 2.035544217687075
